In [1]:
import os
from pathlib import Path
import matlab.engine
import numpy as np
import shutil
import tempfile
import json

# Base directory (adjust to your setup)
base_dir = Path("./")

 # --- Start MATLAB Engine ---
eng = matlab.engine.start_matlab()

 # --- Add SCOPE path to MATLAB's search path ---
scope_path = './SCOPE'  # e.g., '/home/user/Documents/SCOPE'
eng.addpath(scope_path, nargout=0)

# MODTRAN paths
modtran_dir = base_dir / "MODTRAN5"
modtran_exe = modtran_dir / "bin" / "Mod5_mac.exe" 
modtran_tp5_template = modtran_dir / "HyPlant-FLUO_Modtran5_base_v1.tp5"

# SCOPE paths
scope_dir = base_dir / "SCOPE"
scope_main = scope_dir / "SCOPE.m"
scope_wrapper = scope_dir / "run_scope_wrapper.m"

# Output directory
output_dir = base_dir / "synthetic_dataset"
output_dir.mkdir(exist_ok=True)

print(f"Checking path: {scope_dir.resolve()}")
print(f"Directory exists: {scope_dir.exists()}")
print(f"Directory contents: {list(scope_dir.glob('*')) if scope_dir.exists() else 'N/A'}")


Checking path: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/SCOPE
Directory exists: True
Directory contents: [PosixPath('SCOPE/.DS_Store'), PosixPath('SCOPE/run_scope_wrapper_json.m'), PosixPath('SCOPE/input'), PosixPath('SCOPE/output'), PosixPath('SCOPE/docs'), PosixPath('SCOPE/soltir_tp7.m'), PosixPath('SCOPE/SCOPE.m'), PosixPath('SCOPE/README.md'), PosixPath('SCOPE/set_parameter_filenames.csv'), PosixPath('SCOPE/bug_reports.txt'), PosixPath('SCOPE/GNU_General_Public_Licence.txt'), PosixPath('SCOPE/.git'), PosixPath('SCOPE/.readthedocs.yaml'), PosixPath('SCOPE/SCOPE.exe'), PosixPath('SCOPE/src')]


In [ ]:
from scopeWrapper import SCOPEWrapperMultiRun

# Example Usage
if __name__ == "__main__":
    # Diagnostic output
    print("\n=== Path Diagnostics ===")
    print(f"Current working directory: {Path.cwd()}")
    print(f"Checking for 'SCOPE' in: {Path.cwd()}")

    spectral_files = {
        #"Eout_spectrum": "Eout_spectrum.csv",
        #"Lo_spectrum": "Lo_spectrum.csv", 
        "fluorescence": "fluorescence.csv",
        #"sigmaF": "sigmaF.csv",
        "reflectance": "reflectance.csv",
    }
    
    scalar_files = {
        #"aPAR": "aPAR.csv",
        #"Eout": "Eout.csv",
        #"Lo": "Lo.csv",
        #"Esun": "Esun.csv",
        #"Esky": "Esky.csv",
        #"fluxes": "fluxes.csv",
        #"rad": "radiation.csv",
        #"fluorescence_scalars": "fluorescence_scalars.csv",
    }

    try:
        with SCOPEWrapperMultiRun(scalar_files=scalar_files, spectral_files=spectral_files) as scope:
            print("\n=== Running Multi-Simulation Test ===")
            # Example parameters (can be adjusted)

            # Run a simulation with multiple LAI and Cab values
            multi_params = {
                # PROSPECT leaf optical properties
                "Cab": [30.0, 40.0, 50.0],      # Chlorophyll content (μg/cm²)
                "Cca": [10.0],                  # Carotenoid content (μg/cm²)
                "Cdm": [0.012],                 # Dry matter content (g/cm²)
                "Cw": [0.009],                  # Equivalent water thickness (cm)
                "Cs": [0.0],                    # Senescent material fraction (0-1)
                "Cant": [1.0],                  # Anthocyanin content (μg/cm²)
                "N": [1.5],                     # Leaf structure parameter (-)

                # Leaf biochemical parameters
                "Vcmax25": [60.0],              # Maximum carboxylation rate at 25°C (μmol/m²/s)
                "BallBerrySlope": [8.0],        # Ball-Berry stomatal conductance slope
                "BallBerry0": [0.01],           # Ball-Berry intercept (mol/m²/s)

                # Canopy structure
                "LAI": [2.0, 3.0, 5.0],        # Leaf Area Index (m²/m²)
                "hc": [2.0],                    # Canopy height (m)
                "LIDFa": [-0.35],               # Leaf angle distribution parameter a (-)
                "LIDFb": [-0.15],               # Leaf angle distribution parameter b (-)

                # Soil parameters
                "rss": [500.0],                 # Soil respiration rate (μmol/m²/s)
                "SMC": [25.0],                  # Soil moisture content (%)
                
                # Meteorology
                "Rin": [800.0],                 # Incoming shortwave radiation (W/m²)
                "Ta": [20.0],                   # Air temperature (°C)
                "Ca": [410.0],                  # Atmospheric CO₂ concentration (ppm)

                # Fluorescence
                "fqe": [0.01, 0.03, 0.06, 0.1, 0.2], # Fluorescence quantum efficiency (-)

                # Angles
                "tts": [35.0, 0.00, 0.60],                  # Solar zenith angle (degrees)
                "tto": [0.0],                   # Observer zenith angle (degrees)
                "psi": [0.0]                    # Relative azimuth angle (degrees)
            }

            my_setoptions = {
                "simulation": 2,  # Enable Lookup-Table mode
                "calc_fluor": 1,  # Calculate fluorescence
                "soil_heat_method": 2,  # Use soil net radiation fraction
                "saveCSV": 1,  # Output CSV files
                "calc_planck": 0,
                "calc_directional": 0,
                "calc_planck": 0,
                "calc_vert_profiles": 0,
                "calc_rss_rbs": 0,
                "soil_heat_method": 2,
                "save_spectral": 1,
                "calc_xanthophyllabs": 0,
                "applTcorr": 0,
                "MoninObukhov" : 0,
                "lite": 0,
                
            }

            results_path = scope.run(multi_params, my_setoptions)

            print("Success!")
            with open(results_path, "r") as f:
                results = json.load(f)

            print("\n=== Results ===")

            print(
                f"Number of simulations in output: {results.get('num_simulations')}"
            ) 

    except Exception as e:
        print(f"Error during SCOPE execution: {e}")


=== Path Diagnostics ===
Current working directory: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project
Checking for 'SCOPE' in: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project

=== Running Multi-Simulation Test ===
=== Parameters before CSV generation ===
{
  "Cab": [
    30.0,
    40.0,
    50.0
  ],
  "Cca": [
    10.0
  ],
  "Cdm": [
    0.012
  ],
  "Cw": [
    0.009
  ],
  "Cs": [
    0.0
  ],
  "Cant": [
    1.0
  ],
  "N": [
    1.5
  ],
  "Vcmax25": [
    60.0
  ],
  "BallBerrySlope": [
    8.0
  ],
  "BallBerry0": [
    0.01
  ],
  "LAI": [
    2.0,
    3.0,
    5.0
  ],
  "hc": [
    2.0
  ],
  "LIDFa": [
    -0.35
  ],
  "LIDFb": [
    -0.15
  ],
  "rss": [
    500.0
  ],
  "SMC": [
    25.0
  ],
  "Rin": [
    800.0
  ],
  "Ta": [
    20.0
  ],
  "Ca": [
    410.0
  ],
  "fqe": [
    0.01,
    0.03,
    0.06,
    0.1,
    0.2
  ],
  "tts": [
    35.0,
    0.0,
    0.6
  ],
  "tto": [
    0.0
  ],
  "psi": [
    0.0
  ]
}

In [136]:
class SCOPEHandler:
    """Improved SCOPE integration handler with MATLAB engine"""
    
    def __init__(self, scope_path):
        self.scope_path = Path(scope_path).resolve()
        self.eng = None
        self._validate_paths()

    def _validate_paths(self):
        required_files = {
            'main': self.scope_path / 'SCOPE.m',
            'wrapper': self.scope_path / 'run_scope_wrapper_json.m',
            'input_dir': self.scope_path / 'input'
        }
        
        missing = [k for k,v in required_files.items() if not v.exists()]
        if missing:
            raise FileNotFoundError(
                f"Missing required SCOPE files: {', '.join(missing)}\n"
                f"Verify SCOPE installation at: {self.scope_path}"
            )

    def __enter__(self):
        self.eng = matlab.engine.start_matlab('-nodisplay')
        self.eng.addpath(str(self.scope_path), nargout=0)
        self.eng.cd(str(self.scope_path), nargout=0)
        return self

    def __exit__(self, *exc):
        self.eng.quit()
        return False

    def run_simulation(self, parameters):
        """Execute SCOPE with parameter dictionary"""
        with NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
            json.dump(parameters, f)
            temp_json = f.name
        
        try:
            output = self.eng.run_scope_wrapper_json(
                temp_json, 
                str(self.scope_path / 'output' / 'results.json'),
                nargout=1
            )
            return self._parse_output(output)
        finally:
            os.unlink(temp_json)
            shutil.rmtree(self.scope_path / 'output', ignore_errors=True)

    def _parse_output(self, output):
        """Parse MATLAB output structure to Python dict"""
        return {
            'wavelength': np.array(output['wavelength']).ravel(),
            'reflectance': np.array(output['reflectance']).ravel(),
            'fluorescence': np.array(output['fluorescence']).ravel()
        }

In [137]:
import subprocess
from tempfile import NamedTemporaryFile


class MODTRANHandler:
    """MODTRAN simulation handler"""
    
    def __init__(self, modtran_path, template_file):
        self.modtran_path = Path(modtran_path).resolve()
        self.template = self.modtran_path / template_file
        self._validate_setup()

    def _validate_setup(self):
        if not (self.modtran_path / 'bin' / 'Mod5_mac.exe').exists():
            raise FileNotFoundError("MODTRAN executable not found")

    def run_simulation(self, params, case_id):
        """Execute MODTRAN simulation with parameters"""
        tp5_content = self._generate_tp5(params)
        
        with NamedTemporaryFile(mode='w', suffix='.tp5', delete=False) as f:
            f.write(tp5_content)
            tp5_path = Path(f.name)
        
        try:
            subprocess.run(
                [str(self.modtran_path / 'bin' / 'mod5_win64.exe'), str(tp5_path)],
                cwd=self.modtran_path,
                check=True,
                capture_output=True
            )
            return self._parse_output(tp5_path)
        finally:
            tp5_path.unlink()

    def _generate_tp5(self, params):
        """Generate TP5 file from template and parameters"""
        with open(self.template, 'r') as f:
            content = f.read()
        
        replacements = {
            '{H1}': f"{params['sensor_altitude']:.3f}",
            '{SZA}': f"{params['sza']:.1f}",
            '{VIS}': f"{params['visibility']:.2f}",
            # Add more parameter replacements as needed
        }
        
        for ph, val in replacements.items():
            content = content.replace(ph, val)
        
        return content

    def _parse_output(self, tp5_path):
        """Parse MODTRAN output files (.tp7)"""
        tp7_path = tp5_path.with_suffix('.tp7')
        
        # Implement actual parsing based on your MODTRAN output format
        return {
            'direct_transmittance': np.random.rand(100),
            'diffuse_transmittance': np.random.rand(100),
            'spherical_albedo': np.random.rand(100),
            'fluorescence_transmittance': np.random.rand(100)
        }


In [171]:
class SyntheticDataGenerator:
    """Main synthetic dataset generation pipeline"""
    
    def __init__(self, config):
        self.config = config
        self.scope = SCOPEHandler(config['scope_path'])
        self.modtran = MODTRANHandler(
            config['modtran_path'],
            config['modtran_template']
        )
        self.output_dir = Path(config['output_path']).resolve()
        self.output_dir.mkdir(parents=True, exist_ok=True)

    def generate_dataset(self, num_samples=1000):
        """Generate complete synthetic dataset"""
        param_ranges = {
            'lai': (0.5, 6.0),
            'cab': (20, 80),
            'sza': (20, 70),
            'visibility': (5, 50),
            'sensor_altitude': (350, 1500)
        }

        for i in range(num_samples):
            params = self._random_parameters(param_ranges)
            try:
                # Run SCOPE simulation
                with self.scope as scope:
                    scope_result = scope.run_simulation(params)
                
                # Run MODTRAN simulation
                modtran_result = self.modtran.run_simulation(params, i)
                
                # Calculate final radiance
                radiance = self._calculate_radiance(
                    scope_result['reflectance'],
                    scope_result['fluorescence'],
                    modtran_result
                )
                
                # Save sample
                self._save_sample(i, radiance, params)

            except Exception as e:
                print(f"Error generating sample {i}: {str(e)}")
                continue

    def _random_parameters(self, ranges):
        """Generate random parameters within specified ranges"""
        return {
            'lai': np.random.uniform(*ranges['lai']),
            'cab': np.random.uniform(*ranges['cab']),
            'sza': np.random.uniform(*ranges['sza']),
            'visibility': np.random.uniform(*ranges['visibility']),
            'sensor_altitude': np.random.uniform(*ranges['sensor_altitude'])
        }

    def _calculate_radiance(self, reflectance, fluorescence, modtran):
        """Calculate at-sensor radiance using physics model"""
        return (
            (modtran['direct_irradiance'] * modtran['direct_transmittance'] +
            modtran['diffuse_irradiance'] * modtran['diffuse_transmittance'])
            * reflectance / (1 - reflectance * modtran['spherical_albedo'])
        ) + fluorescence * modtran['fluorescence_transmittance']

    def _save_sample(self, idx, radiance, params):
        """Save sample as compressed numpy file"""
        np.savez_compressed(
            self.output_dir / f'sample_{idx:04d}.npz',
            radiance=np.tile(radiance, (17, 17, 1)),  # Create 17x17 patch
            parameters=params,
            wavelength=self.scope.wavelength
        )